<a href="https://www.kaggle.com/code/daskoushik/scratch-mlp?scriptVersionId=114021832" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Activation functions

In [2]:
def sigmoid(Z):
    cache = Z
    A = 1 / (1 + np.exp(-Z))
    return A, cache
def relu(Z):
    cache = Z
    A = np.maximum(0, Z)
    return A, cache
def tanh(Z):
    cache = Z
    A = np.tanh(Z)
    return A, cache
def softmax(Z):
    cache = Z
    A = np.exp(Z - np.max(Z)) / np.exp(Z - np.max(Z)).sum()
    return A, cache

# Model helper functions:

* **initialize_parameters** - initializes at random the parameters weights and biases
* **forward_propagation** - for linear->activation forward prop
* **L_forward_propagation** - for L layers - hidden: linear->activation_hidden, linear->activation_output
* **compute_cost** - cross-entropy loss
* **linear_backward_propagation** - each layer linear back prop
* **activation_backward_propagation** - each layer activation back prop
* **L_backward_propagation** - for L layers back prop
* **update_parameters** - update the parameters

In [3]:
def initialize_parameters(layer_dims):
    """
    Inputs:
    layer_dims - list of number of neurons in each layer
    Returns:
    Wl - Weight matrix of shape (layer_dims[l], layer_dims[l-1])
    b1 - Bias vector of shape (layer_dims[l], 1)
    """
    parameters = {}
    L = len(layer_dims) # total no. of layers
    
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    
    return parameters

In [4]:
def linear_forward_propagation(A, W, b):
    """
    Inputs: 
    A - activation from previous layer
    W - weight, b - bias
    Returns:
    Z - input of activation or pre-activation parameter
    cache - tuple (A, W, b)
    """
    
    Z = np.dot(A, W) + b
    cache = (A, W, b)
    
    return Z, cache

In [5]:
def activation_forward_propagation(A_prev, W, b, activation):
    """
    Inputs:
    A_prev - previous layer activation
    W - weight, b - bias
    Returns:
    A - post-activation
    """
    if activation == "sigmoid":
        Z, linear_cache = linear_forward_propagation(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "tanh":
        Z, linear_cache = linear_forward_propagation(A_prev, W, b)
        A, activation_cache = tanh(Z)
        
    elif activation == "relu":
        Z, linear_cache = linear_forward_propagation(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    elif activation == "softmax":
        Z, linear_cache = linear_forward_propagation(A_prev, W, b)
        A, activation_cache = softmax(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [6]:
def L_forward_propagation(X, parameters, activation_hidden, activation_output):
    """
    Inputs:
    X - input layer
    parameters - weights and biases of L layers
    activation_hidden - hidden layer activation function
    activation_output - output layer activation function
    Returns:
    AL - activation of the output layer
    caches - linear and activation cache to back propagate easily
    """
    caches = []
    A = X
    L = len(parameters) // 2
    
    for l in range(1, L):
        A_prev = A
        if activation_hidden == 'relu':
            A, cache = activation_forward_propagation(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], 'relu')
            caches.append(cache)
        elif activation_hidden == 'tanh':
            A, cache = activation_forward_propagation(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], 'tanh')
            caches.append(cache)
            
    if activation_output == 'softmax':
        AL, cache = activation_forward_propagation(A, parameters['W' + str(L)], parameters['b' + str(L)], 'softmax')
        caches.append(cache)
    elif activation_output == 'sigmoid':
        AL, cache = activation_forward_propagation(A, parameters['W' + str(L)], parameters['b' + str(L)], 'sigmoid')
        caches.append(cache)
    
    return AL, caches

In [7]:
def compute_cost(AL, Y):
    """
    Inputs:
    AL - output layer activation
    Y - true labels
    Returns:
    cost - cross-entropy cost
    """
    
    m = Y.shape[1]
    cost = - (1./m) * (np.dot(Y, np.log(AL).T) + np.dot(1-Y, np.log(1-AL).T))
    cost = np.squeeze(cost)
    
    return cost

In [8]:
def linear_backward_propagation(dZ, cache):
    """
    Inputs:
    dZ - Gradient of the cost with respect to Z
    cache - (A_prev, W, b) from the forward propapagtion
    Returns:
    dA_prev - Gradient of the cost w.r.t. A_prev
    dW - Gradient of the cost w.r.t. W
    db - Gradient of the cost w.r.t. b 
    """
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1./ m * np.dot(dZ, A_prev.T)
    db = 1./ m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [9]:
def activation_backward_propagation(dA, cache, activation):
    """
    Inputs:
    dA - post activation gradient
    cache - (linear_cache, activation_cache)
    activation - activation function to be used
    Returns:
    dA_prev - Gradient of the cost w.r.t. A_prev
    dW - Gradient of the cost w.r.t. W
    db - Gradient of the cost w.r.t. b 
    """
    
    linear_cache, activation_cache = cache
    
    if activation == 'sigmoid':
        s = 1 / (1 + np.exp(-activation_cache))
        dZ = dA * s * (1-s)
        dA_prev, dW, db = linear_backward_propagation(dZ, linear_cache)
    
    elif activation == 'relu':
        dZ = np.array(dA, copy=True)
        dZ[activation_cache <= 0] = 0
        dA_prev, dW, db = linear_backward_propagation(dZ, linear_cache)
    
    elif activation == 'tanh':
        Z = tanh(activation_cache)
        dZ = 1 - Z*Z
        dA_prev, dW, db = linear_backward_propagation(dZ, linear_cache)
    
    elif activation == 'softmax':
        tmp = activation_cache.reshape((-1,1))
        dZ = np.diagflat(activation_cache) - np.dot(tmp, tmp.T)
        dA_prev, dW, db = linear_backward_propagation(dZ, linear_cache)
    
    return dA_prev, dW, db

In [10]:
def L_backward_propagation(AL, Y, caches, activation_hidden, activation_output):
    """
    Inputs:
    AL - ouput of L_model_forward()
    Y - true label vector
    caches - (linear_cache, activation_cache)
    Returns:
    grads - gradients of dAl, dWl, dbl, where l is the layer number
    """
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # Y is same shape as AL
    
    if activation_output == 'sigmoid':
        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        current_cache = caches[L-1]
        dA_prev_temp, dW_temp, db_temp = activation_backward_propagation(dAL, current_cache, 'sigmoid')
        grads["dA" + str(L-1)] = dA_prev_temp
        grads["dW" + str(L)] = dW_temp
        grads["db" + str(L)] = db_temp
    elif activation_output == 'softmax':
        dAL = AL - Y
        current_cache = caches[L-1]
        dA_prev_temp, dW_temp, db_temp = activation_backward_propagation(dAL, current_cache, 'softmax')
        grads["dA" + str(L-1)] = dA_prev_temp
        grads["dW" + str(L)] = dW_temp
        grads["db" + str(L)] = db_temp
        
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = activation_backward_propagation(grads["dA" + str(l+1)], current_cache, activation_hidden)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
    return grads

In [11]:
def update_parameters(params, grads, learning_rate):
    """
    Inputs:
    params - parameters dictionary
    grads - gradients dictionary
    Returns:
    parameters - updated parameters dictionary
    """
    parameters = params.copy()
    L = len(parameters) // 2
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    
    return parameters

# L-layer Model:
Using all the helper functions to create the model

In [12]:
def L_layer_model(X, Y, layers_dims, activation_hidden, activation_output, learning_rate = 0.001, num_iterations=2000, print_cost=False):
    """
    Inputs:
    X - input data matrix
    Y - true labels vector
    layers_dims - list containing the input size and each layer size
    learning_rate - for gradient descent update
    num_iterations - epoch of gradient descent
    print_cost - prints cost of each step
    Returns:
    parameters - learnt parameters by the model
    """
    costs = []
    parameters = initialize_parameters(layers_dims)
    
    for i in range(0, num_iterations):
        # Forward propagation
        AL, caches = L_forward_propagation(X, parameters, activation_hidden, activation_output)
        
        cost = compute_cost(AL, Y)
        
        grads = L_backward_propagation(AL, Y, caches, activation_hidden, activation_output)
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs